CUNY 620 HW2

File imported was from here:
https://snap.stanford.edu/data/email-Eu-core.html

This link lists two files, the below using the file containing list nodes of emails sent from a large european research institution, to other members in the institution.  (The other file contains department labels and was not used.)

The below section of codes reads the file and places them into an list of edges, containing a tuple of source node and destination node.  A second list is created from this, with just the unique source nodes (note this would obviously exclude destination nodes that did not send emails, but excluding those limited nodes, does not seem problematic).

To make this work make sure file email-Eu-core.txt is in the active directory this python script is executed from.

In [51]:
import csv

nodes = []
edges = []

#note file must be in directory script is executing from
f = open("email-Eu-core.txt", "r")

reader = csv.reader(f, delimiter = ' ')
for edge in reader:
    edges.append(edge)
f.close()

edges = [[int(edge[0]), int(edge[1])] for edge in edges]

[nodes.append(edges[i][0]) for i in range(0, len(edges))];
nodes = list(set(nodes))

Below we make a graph out of the list of nodes and add the edges.  Note this graph was not a fully connected graph (i.e., some nodes were isolated from the main set of nodes), since part of the assignment was to explore a diameter of a graph, a subgraph was created, by using the dfs.edges function to walk a path of connected nodes.

After finding the diameter, we then find the dgree of a couple of nodes (i.e., how many edges for each node).

In [60]:
import networkx as nx
import networkx.algorithms.distance_measures as distance
import networkx.classes.function as gFunc
import networkx.algorithms.traversal  as traverse

G = nx.Graph()

[G.add_node(node) for node in nodes];
[G.add_edge(edge[0], edge[1]) for edge in edges];

#find a connected graph...i.e., weed out all nodes that don't connect to node 0, which are only a handful
connectedPath =  list(traverse.dfs_edges(G, 0));

connectedGraph = nx.Graph()
[connectedGraph.add_node(node[0]) for node in connectedPath];

connectedNodes = []
[connectedNodes.append(node[0]) for node in connectedPath];
[connectedGraph.add_edge(edge[0], edge[1]) for edge in edges if edge[0] in connectedNodes];

#this takes a bit ~15-30secs
print("The diameter of the connected graph is: " + str(distance.diameter(connectedGraph)))

gFunc.degree(connectedGraph, [0,1])

The diameter of the connected graph is: 6


DegreeView({0: 44, 1: 48})

Below we print the length of the list of edges.  Note how degree is 1 greater than len of the edges...this is due to edges connecting back to the same node being counted twice (e.g., node 0 has an edge back to node 0).

In [61]:
connectedGraph.edges([0])
print("The length of the list of edges for node 0 is:" + str(len(connectedGraph.edges([0]))))

connectedGraph.edges([1])
print("The length of the list of edges for node 1 is:" + str(len(connectedGraph.edges([1]))))


gFunc.degree(connectedGraph, [0,1])

The length of the list of edges for node 0 is:43
The length of the list of edges for node 1 is:47


DegreeView({0: 44, 1: 48})

Lastly, we write a couple of lines of code to export data of the connected graph that will be used for a neo4j import.

In [59]:
with open('email-sourceIDs.txt', 'w') as f:     
    [f.write(str(node) + '\n') for node in list(connectedGraph.nodes)]
    
with open('email-edges.txt', 'w') as f:     
    [f.write(str(edge[0]) + ',' + str(edge[1])+ '\n') for edge in list(connectedGraph.edges)]    
    
with open('email-edges.txt', 'w') as f:     
    [f.write(str(edge[0]) + ',' + str(edge[1])+ '\n') for edge in list(connectedGraph.edges)]    
    
edgesForNodesGTE700 = []    
[edgesForNodesGTE700.append(edge) for edge in list(connectedGraph.edges())[16000:] if edge[0] >= 700]

with open('email-edges-subset.txt', 'w') as f:     
    [f.write(str(edge[0]) + ',' + str(edge[1])+ '\n') for edge in list(edgesForNodesGTE700)]    
